<a href="https://colab.research.google.com/github/ykitaguchi77/GravCont_classification_2/blob/main/Crop_unilateral_eyelid_olympia_202503.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Crop_unilateral_eyelid_olympia_202503**

```
Google Driveをマウントして、Olympiaデータセットとモデルファイルへのアクセスを設定
Olympia_newデータセットのZIPファイルを解凍
画像ファイルの名前を統一フォーマットに変更（活動期写真は「A」、コントロール写真は「C」のプレフィックスを追加）
事前訓練されたYOLO11lモデルを使用して眼の検出を実行
検出された眼の領域を抽出し、右眼（R）と左眼（L）に分類
抽出された画像を25%拡張して切り抜き、新しいディレクトリに保存
処理された画像をGoogle Driveに転送
```

In [7]:
# prompt: gdriveをマウント

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Olympia_new
zip_path = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/Olympia_new_dataset.zip"

# Heltel_list
Heltel_list_path = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/個人情報なし　コントロールと活動期.xlsx"

# YOLO11l
model_path = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/295+cerebhq1-9655_yolo11l.pt"

In [ ]:
# prompt: zip_pathのzipファイルを解凍

import zipfile
import os

zip_path = zip_path

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content") # Specify the extraction path

print(f"Files extracted!")


Files extracted!


In [ ]:
# prompt: # prompt: dst_dir = "content/Olympia_new/renamed_image"
# # 活動期写真： IMG_0001.JPG →A0001.JPGの名前(4桁0埋め)でdst_dir内にコピー
# # コントロール写真： 1.JPG →C0001.JPGの名前(4桁0埋め)でdst_dir内にコピー

import os
import shutil
from tqdm import tqdm

# Correct the swapped paths according to comments
active_list = "/content/Olympia_new/活動期写真"  # This should be active images
control_list = "/content/Olympia_new/コントロール写真"  # This should be control images
dst_dir = "/content/Olympia_new/renamed_image"

# Create the destination directory if it doesn't exist
if os.path.exists(dst_dir):
    shutil.rmtree(dst_dir)
os.makedirs(dst_dir, exist_ok=True)

print("Processing active images...")
# Process active images with tqdm
active_files = [f for f in os.listdir(active_list) if f.lower().endswith((".jpg", ".jpeg"))]
for filename in tqdm(active_files, desc="Active images"):
    src_path = os.path.join(active_list, filename)
    name, ext = os.path.splitext(filename)
    try:
        # Check if filename is in IMG_XXXX format
        if name.startswith("IMG_"):
            num = int(name[4:])  # Extract the number part
            new_filename = f"A{num:04d}{ext.upper()}"
        else:
            # Try to convert the entire filename to a number
            num = int(name)
            new_filename = f"A{num:04d}{ext.upper()}"
    except ValueError:
        print(f"Skipping file with non-numeric name: {filename}")
        continue
    dst_path = os.path.join(dst_dir, new_filename)
    shutil.copy2(src_path, dst_path)
    #print(f"Copied {filename} to {new_filename}")

print("Processing control images...")
# Process control images with tqdm
control_files = [f for f in os.listdir(control_list) if f.lower().endswith((".jpg", ".jpeg"))]
for filename in tqdm(control_files, desc="Control images"):
    src_path = os.path.join(control_list, filename)
    name, ext = os.path.splitext(filename)
    try:
        # Try to convert the entire filename to a number
        num = int(name)
        new_filename = f"C{num:04d}{ext.upper()}"
    except ValueError:
        print(f"Skipping file with non-numeric name: {filename}")
        continue
    dst_path = os.path.join(dst_dir, new_filename)
    shutil.copy2(src_path, dst_path)

print(f"All images have been processed and saved to {dst_dir}")


Processing active images...


Active images: 100%|██████████| 1037/1037 [00:13<00:00, 75.52it/s] 


Processing control images...


Control images: 100%|██████████| 1034/1034 [00:12<00:00, 85.01it/s]

All images have been processed and saved to /content/Olympia_new/renamed_image


#**Crop eye using pre-trained YOLO11l model**

In [ ]:
!pip install ultralytics --q
!pip install wget --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.1/949.1 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
from ultralytics import YOLO
import wget
import cv2
import matplotlib.pyplot as plt

model_path = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/295+cerebhq1-9655_yolo11l.pt"
model = YOLO(model_path)

# 推論を実行: YOLOの推論
image_path = r'/content/Olympia_new/renamed_image/A0001.JPG'
results = model(image_path, save=False)

for r in results:
    boxes = r.boxes  # YOLOv8の場合

    # YOLO形式出力
    for box in boxes:
        cls_id = int(box.cls[0])  # クラスID
        x_center, y_center, width, height = box.xywhn[0]

        # YOLO形式: class x_center y_center width height
        # 小数点以下6桁で出力したい場合
        print(f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    #検出結果を画像に描画して表示
    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(r.plot(), cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

    # 結果の保存
    #cv2.imwrite('result.jpg', r.plot())

In [ ]:
from ultralytics import YOLO
import wget
import cv2
import numpy as np
import matplotlib.pyplot as plt

model_path = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/295+cerebhq1-9655_yolo11l.pt"
model = YOLO(model_path)

# クラスIDとラベルのマッピング
class_mapping = {0: "R", 1: "L"}  # クラスID 1を"L"にマッピング

# 推論を実行: YOLOの推論
image_path = r'/content/Olympia_new/renamed_image/A0001.JPG'
results = model(image_path, save=False)

# 元の画像を読み込む
original_image = cv2.imread(image_path)
img_height, img_width = original_image.shape[:2]

for r in results:
    boxes = r.boxes  # YOLOv8の場合

    # YOLO形式出力
    for i, box in enumerate(boxes):
        cls_id = int(box.cls[0])  # クラスID
        class_label = class_mapping.get(cls_id, f"Class {cls_id}")  # クラスIDに対応するラベル（R/L）

        # 正規化された座標を取得
        x_center, y_center, width, height = box.xywhn[0].cpu().numpy()

        # 正規化されたYOLO形式を出力
        print(f"{cls_id} ({class_label}) {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

        # 実際のピクセル座標に変換
        x_center_px = x_center * img_width
        y_center_px = y_center * img_height
        width_px = width * img_width
        height_px = height * img_height

        # 拡張された幅を計算 (左右に25%ずつ広げる)
        expanded_width = width_px * 1.5  # 元の幅 + 50% (左右それぞれ25%)

        # 拡張された高さを計算 (拡張された幅と同じに設定)
        expanded_height = expanded_width

        # 新しい中心は元のx中心と同じ、y中心も元のまま
        new_x1 = int(x_center_px - expanded_width / 2)
        new_y1 = int(y_center_px - expanded_height / 2)  # y中心を保持
        new_x2 = int(x_center_px + expanded_width / 2)
        new_y2 = int(y_center_px + expanded_height / 2)  # y中心を保持

        # 範囲外処理のための座標計算
        pad_left = max(0, -new_x1)
        pad_top = max(0, -new_y1)
        pad_right = max(0, new_x2 - img_width)
        pad_bottom = max(0, new_y2 - img_height)

        # 画像内の実際の切り出し座標を計算
        crop_x1 = max(0, new_x1)
        crop_y1 = max(0, new_y1)
        crop_x2 = min(img_width, new_x2)
        crop_y2 = min(img_height, new_y2)

        # 切り出し
        cropped = original_image[crop_y1:crop_y2, crop_x1:crop_x2].copy()

        # パディングが必要な場合、黒で埋める
        if pad_left > 0 or pad_top > 0 or pad_right > 0 or pad_bottom > 0:
            # パディングを含む新しい画像を作成
            padded_width = crop_x2 - crop_x1 + pad_left + pad_right
            padded_height = crop_y2 - crop_y1 + pad_top + pad_bottom
            padded = np.zeros((padded_height, padded_width, 3), dtype=np.uint8)

            # 切り出した部分を黒背景に配置
            padded[pad_top:pad_top+(crop_y2-crop_y1), pad_left:pad_left+(crop_x2-crop_x1)] = cropped
            cropped = padded

        # 切り出した画像を表示（タイトルを修正）
        plt.figure(figsize=(4, 4))
        plt.title(f"{class_label} (ID: {cls_id})", fontsize=10)  # 適切なラベルを表示
        plt.imshow(cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.tight_layout()
        plt.show()

    # 元の検出結果を表示
    result_img = cv2.cvtColor(r.plot(), cv2.COLOR_BGR2RGB)
    original_aspect_ratio = result_img.shape[1] / result_img.shape[0]

    fig_width = 6
    fig_height = fig_width / original_aspect_ratio

    plt.figure(figsize=(fig_width, fig_height))
    plt.title("Original Detection", fontsize=10)
    plt.imshow(result_img)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

###**複数枚一括処理**

Basemameに対して、右眼は_R、左眼は_Lを追加して保存

In [ ]:
from ultralytics import YOLO
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
import shutil
from tqdm import tqdm

# ディレクトリ設定
orig_dir = "/content/Olympia_new/renamed_image"
dst_dir = "/content/Olympia_new/cropped_unilateral_image_25%"

# クラスIDとラベルのマッピング
class_mapping = {0: "R", 1: "L"}

# モデルの読み込み
model_path = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/295+cerebhq1-9655_yolo11l.pt"
model = YOLO(model_path)

# 出力ディレクトリの準備
if os.path.exists(dst_dir):
    shutil.rmtree(dst_dir)  # ディレクトリが存在する場合は削除
os.makedirs(dst_dir)  # ディレクトリを新規作成

# JPGファイルのリストを取得
jpg_files = glob.glob(os.path.join(orig_dir, "*.JPG")) + glob.glob(os.path.join(orig_dir, "*.jpg"))
print(f"Total images found: {len(jpg_files)}")

# tqdmを使用してプログレスバーを表示
for image_path in tqdm(jpg_files, desc="Processing images"):
    # 元のファイル名を取得
    filename = os.path.basename(image_path)
    basename, ext = os.path.splitext(filename)

    # 画像を読み込む
    original_image = cv2.imread(image_path)
    if original_image is None:
        tqdm.write(f"Error: Could not read image {image_path}")
        continue

    img_height, img_width = original_image.shape[:2]

    # 推論を実行
    results = model(image_path, save=False)

    # 各検出結果に対して処理
    detection_count = 0
    for r in results:
        boxes = r.boxes

        for box in boxes:
            cls_id = int(box.cls[0])
            class_label = class_mapping.get(cls_id, f"Class{cls_id}")

            # 正規化された座標を取得
            x_center, y_center, width, height = box.xywhn[0].cpu().numpy()

            # 実際のピクセル座標に変換
            x_center_px = x_center * img_width
            y_center_px = y_center * img_height
            width_px = width * img_width
            height_px = height * img_height

            # 拡張された幅を計算 (左右に25%ずつ広げる)
            expanded_width = width_px * 1.5  # 元の幅 + 50% (左右それぞれ25%)

            # 拡張された高さを計算 (拡張された幅と同じに設定)
            expanded_height = expanded_width

            # 新しい中心は元のx中心と同じ、y中心も元のまま
            new_x1 = int(x_center_px - expanded_width / 2)
            new_y1 = int(y_center_px - expanded_height / 2)
            new_x2 = int(x_center_px + expanded_width / 2)
            new_y2 = int(y_center_px + expanded_height / 2)

            # 範囲外処理のための座標計算
            pad_left = max(0, -new_x1)
            pad_top = max(0, -new_y1)
            pad_right = max(0, new_x2 - img_width)
            pad_bottom = max(0, new_y2 - img_height)

            # 画像内の実際の切り出し座標を計算
            crop_x1 = max(0, new_x1)
            crop_y1 = max(0, new_y1)
            crop_x2 = min(img_width, new_x2)
            crop_y2 = min(img_height, new_y2)

            # 切り出し
            cropped = original_image[crop_y1:crop_y2, crop_x1:crop_x2].copy()

            # パディングが必要な場合、黒で埋める
            if pad_left > 0 or pad_top > 0 or pad_right > 0 or pad_bottom > 0:
                # パディングを含む新しい画像を作成
                padded_width = crop_x2 - crop_x1 + pad_left + pad_right
                padded_height = crop_y2 - crop_y1 + pad_top + pad_bottom
                padded = np.zeros((padded_height, padded_width, 3), dtype=np.uint8)

                # 切り出した部分を黒背景に配置
                padded[pad_top:pad_top+(crop_y2-crop_y1), pad_left:pad_left+(crop_x2-crop_x1)] = cropped
                cropped = padded

            # 保存するファイル名の作成 (basename_R.jpg or basename_L.jpg)
            output_filename = f"{basename}_{class_label}{ext}"
            output_path = os.path.join(dst_dir, output_filename)

            # 切り抜いた画像を保存
            cv2.imwrite(output_path, cropped)
            detection_count += 1

    # 検出されなかった場合に通知
    if detection_count == 0:
        tqdm.write(f"Warning: No eyes detected in {filename}")

print("\nProcessing completed!")
print(f"Processed {len(jpg_files)} images and saved results to {dst_dir}")

In [ ]:
import os
import shutil

# コピー元とコピー先のパス
src_dir = "/content/Olympia_new/cropped_unilateral_image_25%"
dst_dir = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/cropped_unilateral_image_25%"

# コピー先のディレクトリが存在する場合は削除
if os.path.exists(dst_dir):
    print(f"Removing existing directory: {dst_dir}")
    shutil.rmtree(dst_dir)

# ディレクトリごとコピー
print(f"Copying {src_dir} to {dst_dir}...")
shutil.copytree(src_dir, dst_dir)
print("Copy completed successfully!")

# ファイル数を確認
file_count = len([f for f in os.listdir(dst_dir) if os.path.isfile(os.path.join(dst_dir, f))])
print(f"Total files copied: {file_count}")

Copying /content/Olympia_new/cropped_unilateral_image_25% to /content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/cropped_unilateral_image_25%...
Copy completed successfully!
Total files copied: 4142


In [ ]:
import os
import shutil
from tqdm import tqdm

# コピー元とコピー先のパス
src_dir = "/content/Olympia_new/cropped_unilateral_image_25%"
dst_dir = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/cropped_unilateral_image_25%"

# コピー先のディレクトリが存在する場合は削除
if os.path.exists(dst_dir):
    print(f"Removing existing directory: {dst_dir}")
    shutil.rmtree(dst_dir)

# まずディレクトリを作成
os.makedirs(dst_dir)

# ファイル一覧を取得
files = [f for f in os.listdir(src_dir) if os.path.isfile(os.path.join(src_dir, f))]
total_files = len(files)

print(f"Copying {total_files} files from {src_dir} to {dst_dir}...")

# tqdmでファイルをコピー
for file in tqdm(files, desc="Copying files"):
    src_file = os.path.join(src_dir, file)
    dst_file = os.path.join(dst_dir, file)
    shutil.copy2(src_file, dst_file)

print("Copy completed successfully!")
print(f"Total files copied: {total_files}")

Removing existing directory: /content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/cropped_unilateral_image_25%
Copying 4142 files from /content/Olympia_new/cropped_unilateral_image_25% to /content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/cropped_unilateral_image_25%...


Copying files: 100%|██████████| 4142/4142 [01:15<00:00, 55.00it/s]

Copy completed successfully!
Total files copied: 4142


#**Hertelトレーニング用CSV対応表作成**

In [8]:
import pandas as pd

xslx_path = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/個人情報なし　コントロールと活動期.xlsx"


# エクセルファイルのシート名を取得
xls = pd.ExcelFile(xslx_path)
xls.sheet_names

['コントロール', '活動期']

In [11]:
# 再度データを読み込んで、一括処理を行う
xls = pd.ExcelFile(xslx_path)

# "コントロール" シートの処理
df_control = pd.read_excel(xls, sheet_name="コントロール")
df_control_expanded = df_control.melt(
    id_vars=["写真No", "性", "年令"],
    value_vars=["突出度　Ｒ", "突出度　Ｌ"],
    var_name="side",
    value_name="Hertel"
)
df_control_expanded["side"] = df_control_expanded["side"].apply(lambda x: "R" if "Ｒ" in x else "L")
df_control_expanded["Image_path"] = df_control_expanded.apply(lambda row: f"C{row['写真No']:04}_{row['side']}.JPG", axis=1)
df_control_expanded["class"] = 0  # コントロールは 0

# "活動期" シートの処理
df_active = pd.read_excel(xls, sheet_name="活動期").rename(columns={"写真 NO.": "写真No"})
df_active_expanded = df_active.melt(
    id_vars=["写真No", "性", "年令"],
    value_vars=["突出度　Ｒ", "突出度　Ｌ"],
    var_name="side",
    value_name="Hertel"
)
df_active_expanded["side"] = df_active_expanded["side"].apply(lambda x: "R" if "Ｒ" in x else "L")
df_active_expanded["Image_path"] = df_active_expanded.apply(lambda row: f"A{row['写真No']:04}_{row['side']}.JPG", axis=1)
df_active_expanded["class"] = 1  # 活動期は 1

# "写真No" を削除して統合
df_merged = pd.concat([
    df_control_expanded.drop(columns=["写真No"]),
    df_active_expanded.drop(columns=["写真No"])
], ignore_index=True)

# 列名変更と順序変更
df_merged.rename(columns={"性": "sex", "年令": "age"}, inplace=True)
df_merged["sex"] = df_merged["sex"].map({"男": "M", "女": "F"})
df_merged = df_merged[["Image_path", "sex", "age", "side", "Hertel", "class"]]

# 統合後のデータを表示
df_merged

,Image_path,sex,age,side,Hertel,class
0,C0001_R.JPG,F,65,R,11.0,0
1,C0002_R.JPG,F,45,R,15.0,0
2,C0003_R.JPG,F,37,R,13.0,0
3,C0004_R.JPG,F,43,R,13.0,0
4,C0005_R.JPG,F,53,R,14.0,0
...,...,...,...,...,...,...
4137,A1034_L.JPG,F,48,L,15.0,1
4138,A1035_L.JPG,M,57,L,17.0,1
4139,A1036_L.JPG,F,31,L,24.0,1
4140,A1037_L.JPG,F,52,L,20.0,1


In [12]:
# CSVファイルとして保存
csv_file_path = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/Hertel.csv"
df_merged.to_csv(csv_file_path, index=False, encoding="utf-8-sig")

#**Train-val split**

stratified splitにする

In [2]:
orig_dir = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/cropped_unilateral_image_25%"
dst_dir = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/unilateral_25%_trainval"

In [4]:
# """
# ・IDに関してはstratifyしないバージョン
# ・グループ比率維持: AからはじまるID（active）とCからはじまるID（control）の比率が、元のデータセットと同様にtrain/val両方で維持される（stratified split）
# ・分割比率: 全体としてのtrain:val比率は80:20に設定
# """

# import os
# import shutil
# import random
# import numpy as np
# from sklearn.model_selection import train_test_split
# from tqdm.notebook import tqdm  # Colab用にtqdm.notebookを使用

# # Define directories
# orig_dir = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/cropped_unilateral_image_25%"
# dst_dir = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/unilateral_25%_trainval"

# # Remove destination directory if it exists
# if os.path.exists(dst_dir):
#     shutil.rmtree(dst_dir)

# # Create destination directory and its subdirectories
# os.makedirs(dst_dir, exist_ok=True)
# os.makedirs(os.path.join(dst_dir, "train"), exist_ok=True)
# os.makedirs(os.path.join(dst_dir, "val"), exist_ok=True)

# # Get the list of files in the original directory
# files = os.listdir(orig_dir)

# # Separate files starting with 'A' and 'C'
# a_files = [f for f in files if f.startswith('A')]
# c_files = [f for f in files if f.startswith('C')]

# # Set random seed for reproducibility
# random.seed(42)
# np.random.seed(42)

# # Split files into train and validation sets with stratified sampling (80% train, 20% validation)
# a_train, a_val = train_test_split(a_files, test_size=0.2, random_state=42)
# c_train, c_val = train_test_split(c_files, test_size=0.2, random_state=42)

# # Copy files to the destination directories with progress bar
# def copy_files(file_list, destination, desc):
#     for file in tqdm(file_list, desc=desc, position=0, leave=True):
#         src_path = os.path.join(orig_dir, file)
#         dst_path = os.path.join(destination, file)
#         shutil.copy2(src_path, dst_path)
#         # 進捗を確実に更新するため
#         import time
#         time.sleep(0.01)

# # Copy train files
# copy_files(a_train, os.path.join(dst_dir, "train"), "Copying A-files to train")
# copy_files(c_train, os.path.join(dst_dir, "train"), "Copying C-files to train")

# # Copy validation files
# copy_files(a_val, os.path.join(dst_dir, "val"), "Copying A-files to val")
# copy_files(c_val, os.path.join(dst_dir, "val"), "Copying C-files to val")

# # Print statistics
# print(f"Total files: {len(files)}")
# print(f"Files starting with 'A': {len(a_files)}")
# print(f"Files starting with 'C': {len(c_files)}")
# print(f"Train set: {len(a_train) + len(c_train)} files ({len(a_train)} A-files, {len(c_train)} C-files)")
# print(f"Validation set: {len(a_val) + len(c_val)} files ({len(a_val)} A-files, {len(c_val)} C-files)")

Copying A-files to train:   0%|          | 0/1659 [00:00<?, ?it/s]

Copying C-files to train:   0%|          | 0/1654 [00:00<?, ?it/s]

Copying A-files to val:   0%|          | 0/415 [00:00<?, ?it/s]

Copying C-files to val:   0%|          | 0/414 [00:00<?, ?it/s]

Total files: 4142
Files starting with 'A': 2074
Files starting with 'C': 2068
Train set: 3313 files (1659 A-files, 1654 C-files)
Validation set: 829 files (415 A-files, 414 C-files)


In [6]:
"""
・同一ID保存: 同じID（例：A0131）の左右画像ペア（_L.JPGと_R.JPG）は常に同じデータセット（trainまたはval）に配置される
・グループ比率維持: AからはじまるID（active）とCからはじまるID（control）の比率が、元のデータセットと同様にtrain/val両方で維持される（stratified split）
・分割比率: 全体としてのtrain:val比率は80:20に設定
"""

import os
import shutil
import random
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm # Colab用にtqdm.notebookを使用

# Define directories
orig_dir = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/cropped_unilateral_image_25%"
dst_dir = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/unilateral_25%_trainval"

# Remove destination directory if it exists
if os.path.exists(dst_dir):
    print(f"Removing existing directory: {dst_dir}")
    shutil.rmtree(dst_dir)

# Create destination directory and its subdirectories
print(f"Creating directories: {dst_dir}/train and {dst_dir}/val")
os.makedirs(dst_dir, exist_ok=True)
os.makedirs(os.path.join(dst_dir, "train"), exist_ok=True)
os.makedirs(os.path.join(dst_dir, "val"), exist_ok=True)

# Get the list of files in the original directory
print("Reading files from original directory...")
files = os.listdir(orig_dir)

# Group files by ID (removing _L and _R suffix)
file_groups = {}
for file in files:
    # Extract the ID by removing the _L.JPG or _R.JPG suffix
    if '_L.JPG' in file:
        id_key = file.replace('_L.JPG', '')
    elif '_R.JPG' in file:
        id_key = file.replace('_R.JPG', '')
    else:
        id_key = file  # Fallback in case there's a different naming pattern

    if id_key not in file_groups:
        file_groups[id_key] = []
    file_groups[id_key].append(file)

print(f"Found {len(file_groups)} unique IDs")

# Perform stratified split directly on all IDs while preserving A/C ratio
all_ids = list(file_groups.keys())
# Create labels for stratification: 1 for IDs starting with 'A', 0 for 'C'
labels = [1 if id_key.startswith('A') else 0 for id_key in all_ids]

# Count A and C IDs
a_count = sum(labels)
c_count = len(labels) - a_count
print(f"Found {a_count} IDs starting with 'A' and {c_count} IDs starting with 'C'")

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

# Split all IDs into train and validation sets with stratified sampling by A/C label
print("Performing stratified split on all IDs...")
train_ids, val_ids = train_test_split(all_ids, test_size=0.2, stratify=labels, random_state=42)

# Convert ID lists to file lists
train_files = [file for id_key in train_ids for file in file_groups[id_key]]
val_files = [file for id_key in val_ids for file in file_groups[id_key]]

# Count A and C IDs in each split
train_a_count = sum(1 for id_key in train_ids if id_key.startswith('A'))
train_c_count = len(train_ids) - train_a_count
val_a_count = sum(1 for id_key in val_ids if id_key.startswith('A'))
val_c_count = len(val_ids) - val_a_count

# Copy files to the destination directories with tqdm progress bar
def copy_files(file_list, destination, desc):
    for file in tqdm(file_list, desc=desc, position=0, leave=True):
        src_path = os.path.join(orig_dir, file)
        dst_path = os.path.join(destination, file)
        shutil.copy2(src_path, dst_path)

# Copy train and val files
copy_files(train_files, os.path.join(dst_dir, "train"), "Copying files to train")
copy_files(val_files, os.path.join(dst_dir, "val"), "Copying files to val")

# Print statistics
print("\nSplit complete!")
print(f"Total files: {len(files)}")
print(f"Total unique IDs: {len(file_groups)}")
print(f"Original distribution: {a_count} A-IDs ({a_count/len(all_ids)*100:.1f}%), {c_count} C-IDs ({c_count/len(all_ids)*100:.1f}%)")
print(f"Train set: {len(train_files)} files from {len(train_ids)} IDs")
print(f"  - {train_a_count} A-IDs ({train_a_count/len(train_ids)*100:.1f}%)")
print(f"  - {train_c_count} C-IDs ({train_c_count/len(train_ids)*100:.1f}%)")
print(f"Validation set: {len(val_files)} files from {len(val_ids)} IDs")
print(f"  - {val_a_count} A-IDs ({val_a_count/len(val_ids)*100:.1f}%)")
print(f"  - {val_c_count} C-IDs ({val_c_count/len(val_ids)*100:.1f}%)")

Removing existing directory: /content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/unilateral_25%_trainval
Creating directories: /content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/unilateral_25%_trainval/train and /content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/unilateral_25%_trainval/val
Reading files from original directory...
Found 2071 unique IDs
Found 1037 IDs starting with 'A' and 1034 IDs starting with 'C'
Performing stratified split on all IDs...


Copying files to train:   0%|          | 0/3312 [00:00<?, ?it/s]

Copying files to val:   0%|          | 0/830 [00:00<?, ?it/s]


Split complete!
Total files: 4142
Total unique IDs: 2071
Original distribution: 1037 A-IDs (50.1%), 1034 C-IDs (49.9%)
Train set: 3312 files from 1656 IDs
  - 829 A-IDs (50.1%)
  - 827 C-IDs (49.9%)
Validation set: 830 files from 415 IDs
  - 208 A-IDs (50.1%)
  - 207 C-IDs (49.9%)


In [9]:
import os
import glob
import shutil
from PIL import Image
from pathlib import Path
from tqdm import tqdm

# 入力ディレクトリと出力ディレクトリのパス
input_dir = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/unilateral_25%_trainval"
output_dir = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/unilateral_25%_trainval_250px"

# リサイズする画像サイズ
target_size = (250, 250)

def resize_images(input_path, output_path):
    """指定されたディレクトリ内のすべてのJPG画像をリサイズし、同じディレクトリ構造で保存する"""
    # 入力ディレクトリ内のすべてのJPGファイルを検索（大文字小文字の区別なし）
    jpg_files = []
    for ext in ['*.jpg', '*.jpeg', '*.JPG', '*.JPEG']:
        jpg_files.extend(glob.glob(os.path.join(input_path, '**', ext), recursive=True))

    total_files = len(jpg_files)
    print(f"合計 {total_files} 個のJPG画像が見つかりました。")

    # エラーカウンタ
    error_count = 0

    # 進捗バーを表示しながら各画像をリサイズして保存
    for jpg_file in tqdm(jpg_files, desc="画像リサイズ中", unit="枚"):
        # 入力ファイルパスから相対パスを取得
        rel_path = os.path.relpath(jpg_file, input_path)
        # 出力ファイルパスを生成
        output_file = os.path.join(output_path, rel_path)
        # 出力ディレクトリが存在しない場合は作成
        os.makedirs(os.path.dirname(output_file), exist_ok=True)

        try:
            # 画像を開く
            img = Image.open(jpg_file)
            # 画像をリサイズ
            img_resized = img.resize(target_size, Image.LANCZOS)
            # 画像を保存
            img_resized.save(output_file)
        except Exception as e:
            error_count += 1
            print(f"\nエラー ({rel_path}): {str(e)}")

    # 処理結果のサマリーを表示
    print(f"\n処理完了: 合計 {total_files} 枚中 {total_files - error_count} 枚の画像を正常にリサイズしました。")
    if error_count > 0:
        print(f"エラー: {error_count} 枚の画像処理に失敗しました。")

if __name__ == "__main__":
    # 出力ディレクトリが既に存在する場合は削除して作り直す
    if os.path.exists(output_dir):
        print(f"既存の出力ディレクトリを削除します: {output_dir}")
        shutil.rmtree(output_dir)

    # 出力ディレクトリを作成
    os.makedirs(output_dir)
    print(f"新しい出力ディレクトリを作成しました: {output_dir}")

    # リサイズ処理を実行
    resize_images(input_dir, output_dir)
    print("リサイズ処理が完了しました。")

既存の出力ディレクトリを削除します: /content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/unilateral_25%_trainval_250px
新しい出力ディレクトリを作成しました: /content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/unilateral_25%_trainval_250px
合計 4142 個のJPG画像が見つかりました。


画像リサイズ中: 100%|██████████| 4142/4142 [03:58<00:00, 17.37枚/s]


処理完了: 合計 4142 枚中 4142 枚の画像を正常にリサイズしました。
リサイズ処理が完了しました。
